# 1. Data Preprocessing

## 1.1 Import Dependencies

In [22]:
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from collections import defaultdict
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [2]:
for dirname, _, filenames in os.walk('./data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

./data/test_round2.csv
./data/train_round2.csv
./data/graph_round2.csv
./data/test.csv
./data/graph.csv
./data/submission_round2.csv
./data/submission.csv
./data/train.csv


## 1.2 Load Data

In [3]:
data_dir = 'data'
temporal_data_path = os.path.join(data_dir, 'train.csv')
mobility_data_path = os.path.join(data_dir, 'graph_round2.csv')

temporal_data = pd.read_csv(temporal_data_path)
temporal_data.describe()
# train_round2.csv 04/12/2020 to 11/22/2020.
# train.csv 04/12/2020 to 08/31/2020

,ID,Confirmed,Deaths,Recovered,Active,Incident_Rate,People_Tested,People_Hospitalized,Mortality_Rate,Testing_Rate,Hospitalization_Rate
count,11250.000000,1.125000e+04,11250.000000,9419.000000,1.125000e+04,11250.000000,1.055000e+04,5047.000000,10550.000000,11250.000000,5047.000000
mean,5624.500000,9.583625e+04,3057.178311,40276.189086,5.907248e+04,1357.938709,1.161268e+06,6250.044185,3.300561,20085.845676,12.197144
std,3247.739599,1.516621e+05,5377.615328,81746.843553,1.243728e+05,1199.193935,2.031813e+06,13880.694435,2.039538,19319.956138,5.255183
min,0.000000,2.700000e+02,0.000000,0.000000,-9.890000e+02,32.771004,5.459000e+03,10.000000,0.000000,460.300152,2.302896
25%,2812.250000,1.054600e+04,273.000000,3030.500000,4.599500e+03,374.587118,1.579640e+05,595.000000,1.708176,5761.788808,8.387037
50%,5624.500000,4.175800e+04,1031.000000,10376.000000,1.579100e+04,1045.850447,4.901655e+05,2068.000000,2.802207,15172.226199,11.351161
75%,8436.750000,1.174958e+05,3387.000000,48028.000000,5.289675e+04,2046.182244,1.247765e+06,6134.500000,4.425834,27209.961598,15.396088
max,11249.000000,1.153529e+06,34319.000000,913796.000000,1.095798e+06,9537.675412,1.956515e+07,89995.000000,9.741481,134755.702292,38.501190


In [4]:
states = list(np.unique(temporal_data['Province_State']))
state_cum_temporal_data = dict.fromkeys(states, None)
state_temporal_data = dict.fromkeys(states, None)
# state_cum_temporal_scaler = defaultdict(StandardScaler)
# state_temporal_scaler = defaultdict(StandardScaler)
dropped_attr = ['Date',
                'Active',
                'ID', 
                'Province_State', 
                'Incident_Rate', 
                'Recovered', 
                'People_Tested', 
                'People_Hospitalized', 
                'Mortality_Rate', 
                'Testing_Rate', 
                'Hospitalization_Rate']

for s in states:
    df_filter = temporal_data['Province_State'] == s
    state_df = temporal_data[df_filter]
    # Daily difference data
    state_temporal_data[s] = state_df.drop(dropped_attr, 1)
    for col in state_temporal_data[s]:
        data = state_temporal_data[s][col].tolist()
        diff = [i - j for i, j in zip(data, [*[data[0]], *data[:len(data) - 1]])]
        state_temporal_data[s][col] = diff
    save_columns = state_temporal_data[s].columns
#     state_temporal_data[s] = state_temporal_scaler[s].fit_transform(state_temporal_data[s])
    state_temporal_data[s] = pd.DataFrame(state_temporal_data[s], columns=save_columns)
    # Daily cumulative data
    state_cum_temporal_data[s] = state_df.drop(dropped_attr, 1)
    save_columns = state_cum_temporal_data[s].columns
#     state_cum_temporal_data[s] = state_cum_temporal_scaler[s].fit_transform(state_cum_temporal_data[s])
    state_cum_temporal_data[s] = pd.DataFrame(state_cum_temporal_data[s], columns=save_columns)
    
print(state_temporal_data['California'])
print(state_cum_temporal_data['California'])

       Confirmed  Deaths
4              0       0
54          1136      74
104         1425      53
154         1330      93
204          991      96
...          ...     ...
11004      11755     106
11054      13134      89
11104      12576      98
11154      15685      32
11204       9089      54

[225 rows x 2 columns]
       Confirmed  Deaths
4          22795     640
54         23931     714
104        25356     767
154        26686     860
204        27677     956
...          ...     ...
11004    1064040   18453
11054    1077174   18542
11104    1089750   18640
11154    1105435   18672
11204    1114524   18726

[225 rows x 2 columns]


## Train a RNN Model with LSTM

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras import Sequential
from tensorflow.keras import Input
from tensorflow.keras.models import Model

# window_size is use n-1 days to predict nth day
window_size = 22
forecast_days = 26

# test model for current_state
current_state = 'Georgia'
data_training = state_temporal_data[current_state]

In [ ]:
scaler = MinMaxScaler()
data_training = scaler.fit_transform(data_training)

In [ ]:
X_train = []
y_train = []

data_training_np = np.array(data_training)

for i in range(data_training.shape[0] - window_size):
    X_train.append(data_training[i : i + window_size])
    y_train.append(data_training_np[i + window_size])
    
X_train, y_train = np.array(X_train), np.array(y_train)

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
model = keras.Sequential()

model.add(LSTM(units=10, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.1))

model.add(LSTM(units=20, return_sequences=False))
model.add(Dropout(0.1))

model.add(Dense(units = 2))

model.summary()

In [ ]:
model.compile(optimizer='adam', loss = 'mean_squared_logarithmic_error')
model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=1)

In [ ]:
prediction_queue = X_train[-1:]
prediction_queue = np.array(prediction_queue)
# prediction_queue

In [ ]:
# prediction_queue should take in this y_pred as the last day and pop the first day in the queue
# use model.predict(prediction_queue) with the new prediction_queue to get second day.
# need to predict for next forecast_days days.
for i in range(forecast_days):  
    y_pred = model.predict(prediction_queue)
    prediction_queue = np.append(prediction_queue, y_pred)
    prediction_queue = np.delete(prediction_queue, 0)
    prediction_queue = np.delete(prediction_queue, 0)
    prediction_queue = prediction_queue.reshape(1,int(prediction_queue.shape[0]/2),2)

In [ ]:
y_case_forecast = []
y_death_forecast = []

for i in range(forecast_days):
    y_case_forecast.append(prediction_queue[0][i + window_size - forecast_days][0])
    y_death_forecast.append(prediction_queue[0][i + window_size - forecast_days][1])

In [ ]:
case_scale = 1/scaler.scale_[0]
death_scale = 1/scaler.scale_[1]

In [ ]:
y_case_forecast = [case_scale * i for i in y_case_forecast]
y_death_forecast = [death_scale * i for i in y_death_forecast]

In [ ]:
last_day_case = state_cum_temporal_data[current_state]['Confirmed'].iloc[-1]
last_day_death = state_cum_temporal_data[current_state]['Deaths'].iloc[-1]

y_cum_case_forecast = []
y_cum_death_forecast = []

cum_case = last_day_case
cum_death = last_day_death
for case in y_case_forecast:
    cum_case += case
    y_cum_case_forecast.append(cum_case)
    
for death in y_death_forecast:
    cum_death += death
    y_cum_death_forecast.append(cum_death)

In [ ]:
y_cum_case_forecast

In [ ]:
y_cum_death_forecast

# 2. Train a RNN with LSTM for Every State

## 2.1 Import Dependencies

In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras import Sequential
from tensorflow.keras import Input
from tensorflow.keras.models import Model

# window_size is use n-1 days to predict nth day
window_size = 22
forecast_days = 26

## 2.2 Helper Functions

In [8]:
states = state_temporal_data.keys()

def get_training_data(state):
    state_temporal_train = state_temporal_data[state]
    scaler = MinMaxScaler()
    state_temporal_train = scaler.fit_transform(state_temporal_train)
    
    X_train = []
    y_train = []

    state_temporal_train_np = np.array(state_temporal_train)

    for i in range(state_temporal_train.shape[0] - window_size):
        X_train.append(state_temporal_train[i : i + window_size])
        y_train.append(state_temporal_train_np[i + window_size])

    return np.array(X_train), np.array(y_train), scaler
    

def get_model(X_train):
    model = keras.Sequential()

    model.add(LSTM(units=10, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dropout(0.1))

    model.add(LSTM(units=20, return_sequences=False))
    model.add(Dropout(0.1))

    model.add(Dense(units = 2))
    
    return model


def get_prediction_queue(X_train):
    prediction_queue = X_train[-1:]
    return np.array(prediction_queue)


def predict(model, queue, scaler):
    prediction_queue = queue
    
    for i in range(forecast_days):  
        y_pred = model.predict(prediction_queue)
        prediction_queue = np.append(prediction_queue, y_pred)
        prediction_queue = np.delete(prediction_queue, 0)
        prediction_queue = np.delete(prediction_queue, 0)
        prediction_queue = prediction_queue.reshape(1, prediction_queue.shape[0] // 2, 2)
        
    y_case_forecast = []
    y_death_forecast = []

    for i in range(forecast_days):
        y_case_forecast.append(prediction_queue[0][i + window_size - forecast_days][0])
        y_death_forecast.append(prediction_queue[0][i + window_size - forecast_days][1])
        
    case_scale = 1 / scaler.scale_[0]
    death_scale = 1 / scaler.scale_[1]
    
    return [case_scale * i for i in y_case_forecast], [death_scale * i for i in y_death_forecast]


def get_cum_forecast(state, y_case_forecast, y_death_forecast):
    last_day_case = state_cum_temporal_data[state]['Confirmed'].iloc[-1]
    last_day_death = state_cum_temporal_data[state]['Deaths'].iloc[-1]

    y_cum_case_forecast = []
    y_cum_death_forecast = []

    cum_case = last_day_case
    cum_death = last_day_death
    for case in y_case_forecast:
        cum_case += case
        y_cum_case_forecast.append(cum_case)

    for death in y_death_forecast:
        cum_death += death
        y_cum_death_forecast.append(cum_death)
        
    return y_cum_case_forecast, y_cum_death_forecast

## 2.3 Train and Predict

In [9]:
forecasted = {}

for state in states:
    X_train, y_train, scaler = get_training_data(state)

    model = get_model(X_train)
    model.compile(optimizer='adam', loss = 'mean_squared_logarithmic_error')
    model.fit(X_train, y_train, epochs=50, batch_size=32)
    
    y_case_forecast, y_death_forecast = predict(model, get_prediction_queue(X_train), scaler)
    y_cum_case_forecast, y_cum_death_forecast = get_cum_forecast(state, y_case_forecast, y_death_forecast)
    
    forecasted[state] = (y_cum_case_forecast, y_cum_death_forecast)

Epoch 1/50
7/7 [==============================] - 0s 16ms/step - loss: 0.0627
Epoch 2/50
7/7 [==============================] - 0s 10ms/step - loss: 0.0500
Epoch 3/50
7/7 [==============================] - 0s 10ms/step - loss: 0.0505
Epoch 4/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0493
Epoch 5/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0482
Epoch 6/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0467
Epoch 7/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0328
Epoch 8/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0152
Epoch 9/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0126
Epoch 10/50
7/7 [==============================] - 0s 10ms/step - loss: 0.0126
Epoch 11/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0113
Epoch 12/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0104
Epoch 13/50
7/7 [==============================] - 0s 9ms/step - loss

7/7 [==============================] - 0s 9ms/step - loss: 0.0134
Epoch 11/50
7/7 [==============================] - 0s 10ms/step - loss: 0.0132
Epoch 12/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0126
Epoch 13/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0117
Epoch 14/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0119
Epoch 15/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0116
Epoch 16/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0111
Epoch 17/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0114
Epoch 18/50
7/7 [==============================] - 0s 10ms/step - loss: 0.0117
Epoch 19/50
7/7 [==============================] - 0s 10ms/step - loss: 0.0119
Epoch 20/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0115
Epoch 21/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0113
Epoch 22/50
7/7 [==============================] - 0s 9ms/step - loss: 0.

7/7 [==============================] - 0s 9ms/step - loss: 0.0040
Epoch 16/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0044
Epoch 17/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0045
Epoch 18/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0043
Epoch 19/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0042
Epoch 20/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0044
Epoch 21/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0040
Epoch 22/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0042
Epoch 23/50
7/7 [==============================] - 0s 10ms/step - loss: 0.0043
Epoch 24/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0041
Epoch 25/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0041
Epoch 26/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0044
Epoch 27/50
7/7 [==============================] - 0s 9ms/step - loss: 0.00

7/7 [==============================] - 0s 9ms/step - loss: 0.0109
Epoch 21/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0102
Epoch 22/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0096
Epoch 23/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0105
Epoch 24/50
7/7 [==============================] - 0s 10ms/step - loss: 0.0104
Epoch 25/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0105
Epoch 26/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0102
Epoch 27/50
7/7 [==============================] - 0s 10ms/step - loss: 0.0101
Epoch 28/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0101
Epoch 29/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0102
Epoch 30/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0091
Epoch 31/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0098
Epoch 32/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0

7/7 [==============================] - 0s 9ms/step - loss: 0.0075
Epoch 26/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0075
Epoch 27/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0071
Epoch 28/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0071
Epoch 29/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0078
Epoch 30/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0070
Epoch 31/50
7/7 [==============================] - 0s 10ms/step - loss: 0.0076
Epoch 32/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0073
Epoch 33/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0070
Epoch 34/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0064
Epoch 35/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0072
Epoch 36/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0073
Epoch 37/50
7/7 [==============================] - 0s 9ms/step - loss: 0.00

...

7/7 [==============================] - 0s 8ms/step - loss: 0.0107
Epoch 31/50
7/7 [==============================] - 0s 8ms/step - loss: 0.0105
Epoch 32/50
7/7 [==============================] - 0s 8ms/step - loss: 0.0105
Epoch 33/50
7/7 [==============================] - 0s 8ms/step - loss: 0.0106
Epoch 34/50
7/7 [==============================] - 0s 8ms/step - loss: 0.0106
Epoch 35/50
7/7 [==============================] - 0s 8ms/step - loss: 0.0102
Epoch 36/50
7/7 [==============================] - 0s 8ms/step - loss: 0.0106
Epoch 37/50
7/7 [==============================] - 0s 8ms/step - loss: 0.0103
Epoch 38/50
7/7 [==============================] - 0s 8ms/step - loss: 0.0101
Epoch 39/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0102
Epoch 40/50
7/7 [==============================] - 0s 8ms/step - loss: 0.0111
Epoch 41/50
7/7 [==============================] - 0s 8ms/step - loss: 0.0104
Epoch 42/50
7/7 [==============================] - 0s 8ms/step - loss: 0.011

7/7 [==============================] - 0s 9ms/step - loss: 0.0119
Epoch 41/50
7/7 [==============================] - 0s 11ms/step - loss: 0.0120
Epoch 42/50
7/7 [==============================] - 0s 17ms/step - loss: 0.0118
Epoch 43/50
7/7 [==============================] - 0s 13ms/step - loss: 0.0111
Epoch 44/50
7/7 [==============================] - 0s 11ms/step - loss: 0.0113
Epoch 45/50
7/7 [==============================] - 0s 11ms/step - loss: 0.0120
Epoch 46/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0110
Epoch 47/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0118
Epoch 48/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0110
Epoch 49/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0107
Epoch 50/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0108
Epoch 1/50
7/7 [==============================] - 0s 13ms/step - loss: 0.0289
Epoch 2/50
7/7 [==============================] - 0s 10ms/step - loss: 

7/7 [==============================] - 0s 10ms/step - loss: 0.0087
Epoch 46/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0090
Epoch 47/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0090
Epoch 48/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0084
Epoch 49/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0085
Epoch 50/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0084
Epoch 1/50
7/7 [==============================] - 0s 13ms/step - loss: 0.0394
Epoch 2/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0298
Epoch 3/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0145
Epoch 4/50
7/7 [==============================] - 0s 10ms/step - loss: 0.0091
Epoch 5/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0087
Epoch 6/50
7/7 [==============================] - 0s 10ms/step - loss: 0.0083
Epoch 7/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0076
E

7/7 [==============================] - 0s 10ms/step - loss: 0.0050
Epoch 1/50
7/7 [==============================] - 0s 12ms/step - loss: 0.0228
Epoch 2/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0094
Epoch 3/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0089
Epoch 4/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0077
Epoch 5/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0071
Epoch 6/50
7/7 [==============================] - 0s 10ms/step - loss: 0.0066
Epoch 7/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0060
Epoch 8/50
7/7 [==============================] - 0s 10ms/step - loss: 0.0061
Epoch 9/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0053
Epoch 10/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0050
Epoch 11/50
7/7 [==============================] - 0s 8ms/step - loss: 0.0045
Epoch 12/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0042
Epo

7/7 [==============================] - 0s 10ms/step - loss: 0.0290
Epoch 6/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0277
Epoch 7/50
7/7 [==============================] - 0s 10ms/step - loss: 0.0285
Epoch 8/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0279
Epoch 9/50
7/7 [==============================] - 0s 11ms/step - loss: 0.0285
Epoch 10/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0279
Epoch 11/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0276
Epoch 12/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0277
Epoch 13/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0280
Epoch 14/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0279
Epoch 15/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0276
Epoch 16/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0280
Epoch 17/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0277

## 2.4 Generate Submission File

In [11]:
forecastID = [x for x in range(forecast_days * 50)]
deaths = []
confirmed = []

for i in range(forecast_days):
    for s in states:
        print(f"Day {i+1}/26, {s}")
        confirmed.append(forecasted[s][0][i])
        deaths.append(forecasted[s][1][i])
        
final = pd.DataFrame(list(zip(forecastID, confirmed, deaths)), 
               columns =['ForecastID', 'Confirmed', 'Deaths']) 

final.to_csv("submission.csv", index=False)

Day 0/7, Alabama
Day 0/7, Alaska
Day 0/7, Arizona
Day 0/7, Arkansas
Day 0/7, California
Day 0/7, Colorado
Day 0/7, Connecticut
Day 0/7, Delaware
Day 0/7, Florida
Day 0/7, Georgia
Day 0/7, Hawaii
Day 0/7, Idaho
Day 0/7, Illinois
Day 0/7, Indiana
Day 0/7, Iowa
Day 0/7, Kansas
Day 0/7, Kentucky
Day 0/7, Louisiana
Day 0/7, Maine
Day 0/7, Maryland
Day 0/7, Massachusetts
Day 0/7, Michigan
Day 0/7, Minnesota
Day 0/7, Mississippi
Day 0/7, Missouri
Day 0/7, Montana
Day 0/7, Nebraska
Day 0/7, Nevada
Day 0/7, New Hampshire
Day 0/7, New Jersey
Day 0/7, New Mexico
Day 0/7, New York
Day 0/7, North Carolina
Day 0/7, North Dakota
Day 0/7, Ohio
Day 0/7, Oklahoma
Day 0/7, Oregon
Day 0/7, Pennsylvania
Day 0/7, Rhode Island
Day 0/7, South Carolina
Day 0/7, South Dakota
Day 0/7, Tennessee
Day 0/7, Texas
Day 0/7, Utah
Day 0/7, Vermont
Day 0/7, Virginia
Day 0/7, Washington
Day 0/7, West Virginia
Day 0/7, Wisconsin
Day 0/7, Wyoming
Day 1/7, Alabama
Day 1/7, Alaska
Day 1/7, Arizona
Day 1/7, Arkansas
Day 1/7, C